# Advent of Code

## --- Day 8: Treetop Tree House ---

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:

30373
25512
65332
33549
35390

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

    The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
    The top-middle 5 is visible from the top and right.
    The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
    The left-middle 5 is visible, but only from the right.
    The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
    The right-middle 3 is visible from the right.
    In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

To begin, [get your puzzle input](https://adventofcode.com/2022/day/8/input).

Answer:



In [1]:
data = open("aoc_data_08.txt").read().strip().split("\n")
#data = ["30373", "25512", "65332", "33549", "35390"]

In [2]:
import numpy as np

N = len(data)
S = 4 * (N - 1)
n = len(data) - 2

A = np.zeros([N,N])
for i in range(N):
    for j in range(N):
        A[i][j] = int(data[i][j])

In [3]:
print(A)
print(N)

[[0. 0. 0. ... 2. 2. 2.]
 [2. 2. 1. ... 2. 2. 0.]
 [1. 2. 2. ... 2. 0. 2.]
 ...
 [0. 2. 0. ... 0. 1. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 [0. 2. 2. ... 1. 2. 1.]]
99


In [4]:
R1 = np.zeros((n,n))

for i in range(n):
    for j in range(n):
        #print(i + 1, j + 1, ":", A[i + 1, j + 1])
        R1[i,j] = sum([
                np.greater_equal(A[i + 1, 0:(j + 1)], np.full((1, j + 1),A[i + 1, j + 1])).sum() == 0,
                np.greater_equal(A[i + 1, (j+2):], np.full((1, N - j - 2),A[i + 1,j + 1])).sum() == 0,
                np.greater_equal(A[0:(i + 1), (j + 1)],  np.full((1, i + 1), A[i + 1,j + 1])).sum() == 0,
                np.greater_equal(A[(i + 2):, (j + 1)], np.full((1, N - i - 2), A[i + 1,j + 1])).sum() == 0
                ]
            ) > 0
        #print(R[i,j])
        #print()

R1.sum() + S

1785.0

Your puzzle answer was 1785.

## --- Part Two ---

Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

30373
25512
65332
33549
35390

    Looking up, its view is not blocked; it can see 1 tree (of height 3).
    Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
    Looking right, its view is not blocked; it can see 2 trees.
    Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).

A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

30373
25512
65332
33549
35390

    Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
    Looking left, its view is not blocked; it can see 2 trees.
    Looking down, its view is also not blocked; it can see 1 tree.
    Looking right, its view is blocked at 2 trees (by a massive tree of height 9).

This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?

In [5]:
%%time
R2 = np.ones((n,n))

for i in range(n):
    for j in range(n):

        a = np.greater_equal(A[i + 1, 0:(j + 1)], np.full((1, j + 1),A[i + 1, j + 1])[0]) # Left
        b = a[::-1]
        if np.nonzero(b)[0].size > 0:
            score_left =  np.min(np.where(b)) + 1
        else:
            score_left = b.size

        a = np.greater_equal(A[i + 1, (j+2):], np.full((1, N - j - 2),A[i + 1,j + 1])[0]) # Right
        if np.nonzero(a)[0].size > 0:
            score_right =  np.min(np.where(a)) + 1
        else:
            score_right = a.size


        a  = np.greater_equal(A[0:(i + 1), (j + 1)],  np.full((1, i + 1), A[i + 1,j + 1])[0]) #Top
        b = a[::-1]
        if np.nonzero(b)[0].size > 0:
            score_top =  np.min(np.where(b)) + 1
        else:
            score_top = b.size


        a = np.greater_equal(A[(i + 2):, (j + 1)], np.full((1, N - i - 2), A[i + 1,j + 1])[0]) # Bottom
        if np.nonzero(a)[0].size > 0:
            score_bottom =  np.min(np.where(a)) + 1
        else:
            score_bottom = a.size

        score = score_top * score_bottom * score_left * score_right
        R2 [i,j] = score

R2.max()

CPU times: user 255 ms, sys: 570 µs, total: 255 ms
Wall time: 256 ms


345168.0

Your puzzle answer was 345168.